# Setting up

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ta
# from fastai import *
# from fastai.tabular import *
from tqdm import tqdm_notebook
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LassoLarsCV
from rolling import RollingWindowSplit
from sklearn.metrics import r2_score as r2d2
from joblib import dump, load
from datetime import datetime, timedelta

%matplotlib inline
%load_ext line_profiler
sns.set(style = "whitegrid")
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [ ]:
# %%time
# path = 'D://Coding//XTX Forecasting Challenge//data-training.csv'
# df = pd.read_csv(path)

In [2]:
path = 'D://Coding//XTX Forecasting Challenge//data-training.file'
df = pd.read_feather(path, use_threads=8)
df = df.astype('float32')
df.fillna(0, inplace=True)

In [3]:
askRateList = ['askRate' + str(i) for i in range(0,15)]
askSizeList = ['askSize' + str(i) for i in range(0,15)]
bidRateList = ['bidRate' + str(i) for i in range(0,15)]
bidSizeList = ['bidSize' + str(i) for i in range(0,15)]

In [4]:
relevant = ['totalBidVol1', 'totalAskVol1', 'totalBidVol2', 'totalAskVol2',
           'totalBidVol3', 'totalAskVol3', 'totalBidVol4', 'totalAskVol4',
           'totalBidVol5', 'totalAskVol5', 'bidAskRatio4', 'bidAskRatio5', 'OIR',
           'daskRate6', 'dbidRate6', 'daskRate7', 'dbidRate7', 'daskRate8',
           'dbidRate8', 'daskRate9', 'dbidRate9', 'daskRate10', 'dbidRate10',
           'daskRate20', 'midRate', 'bidAskVol', 'others_dlr']

# Exploratory Data Analysis

In [ ]:
# # Figuring out what [y] is
# # y(t) is midRate(t+87) - midRate(t), clipped to (-5.5)
# df['expectedY'] = df.midRate.diff(87).shift(-87).clip(-5,5)

# Feature engineering

#### Cross-sectional features

In [ ]:
# different from submission
def compute_cross_sectional(df):
    # Cross-sectional features
    df['spread'] = df.askRate0 - df.bidRate0
    df['midRate'] = (df.askRate0 + df.bidRate0) / 2
    df['bidAskVol'] = df.askSize0 + df.bidSize0
    df['totalBidVol1'] = df.bidSize0 + df.bidSize1
    df['totalAskVol1'] = df.askSize0 + df.askSize1
    for i in range(2,15):
        df['totalBidVol' + str(i)] = df['totalBidVol' + str(i-1)] + df['bidSize' + str(i)]
        df['totalAskVol' + str(i)] = df['totalAskVol' + str(i-1)] + df['askSize' + str(i)]
    for i in range(1,15):
        df['bidAskRatio' + str(i)] = df['totalBidVol' + str(i)] / df['totalAskVol' + str(i)]
    df['totalAvailVol'] = df.totalBidVol14 + df.totalAskVol14
    df['vwaBid'] = np.einsum('ij,ji->i', df[bidRateList], df[bidSizeList].T) / df[bidSizeList].sum(axis=1)
    df['vwaAsk'] = np.einsum('ij,ji->i', df[askRateList], df[askSizeList].T) / df[askSizeList].sum(axis=1)
    df['vwaBidDMid'] = df.midRate - df.vwaBid
    df['vwaAskDMid'] = df.vwaAsk - df.midRate
    df['diff_vwaBidAskDMid'] = df.vwaAskDMid - df.vwaBidDMid
    return df

#### Time series features

In [ ]:
def add_time_features(df):
    b1, a1 = (df.bidRate0 < df.bidRate0.shift(1)), (df.askRate0 < df.askRate0.shift(1))
    b2, a2 = (df.bidRate0 == df.bidRate0.shift(1)), (df.askRate0 == df.askRate0.shift(1))
    valsB, valsA = [0, (df.bidSize0 - df.bidSize0.shift(1))], [0, (df.askSize0 - df.askSize0.shift(1))]
    defaultB, defaultA = df.bidSize0, df.askSize0
    df.fillna(0, inplace=True)
    df['deltaVBid'] = np.select([b1,b2], valsB, default=defaultB)
    df['deltaVAsk'] = np.select([a1,a2], valsA, default=defaultA)
    df['VOI'] = df.deltaVBid - df.deltaVAsk
    df['OIR'] = (df.bidSize0 - df.askSize0)/(df.bidSize0 + df.askSize0)
    return df

#### Manual time features — can consider adding more to the lags list

In [ ]:
def add_manual_time_features(df):
    lags = [*np.arange(1,10), *np.arange(10,100,10), *np.arange(100,1000,100)]
    def addTimeFeatures(i):
        df['daskRate' + str(i)] = df.askRate0.diff(i)
        df['dbidRate' + str(i)] = df.bidRate0.diff(i)
    for i in lags:
        addTimeFeatures(i)
    df.fillna(0, inplace=True)
    return df

In [ ]:
df = compute_cross_sectional(df)
df = add_time_features(df)
df = add_manual_time_features(df)

In [ ]:
# df.to_feather('intermediate.file')
df = pd.read_feather('intermediate.file', use_threads=8)

#### Tick chart version with ffill

In [ ]:
# midrate version
df['time'] = pd.date_range(start='1/1/1970', periods=2999999, freq='T')
df.set_index('time', inplace=True)
df_mid = df.midRate.resample('15Min').ohlc()
df_mid['vol'] = df.bidAskVol.resample('15Min').mean()

In [ ]:
df_mid_ta = ta.add_all_ta_features(df_mid, "open", "high", "low", "close", "vol", fillna=True)

In [ ]:
# takes 30s
new_df = df.join(df_mid_ta).ffill().astype('float32')

In [ ]:
# dump(new_df, 'new_df.joblib')
new_df = load('new_df.joblib')

# Cross-validation

In [ ]:
# the leaderboard set should have 150k: they check running time of 10k in 1h and max 15h

In [13]:
# 70-30 train-valid and test split
def train_test_split(df):
#     test_start = int(0.95*len(df))
    test_start = -100
    train_df = df[:test_start].copy()
    test_df = df[test_start:].copy()
    return train_df, test_df
train_df, test_df = train_test_split(df)
# train_df, test_df = train_test_split(new_df)

In [ ]:
def create_limited_features_orig(df):
    df['midRate'] = (df.bidRate0 + df.askRate0) / 2 # necessary for ohlc
    df['bidAskVol'] = df.bidSize0 + df.askSize0 # necessary only for volume_adi
    df['totalBidVol1'] = df.bidSize0 + df.bidSize1
    df['totalAskVol1'] = df.askSize0 + df.askSize1
    for i in range(2,6):
        df['totalBidVol' + str(i)] = df['totalBidVol' + str(i-1)] + df['bidSize' + str(i)]
        df['totalAskVol' + str(i)] = df['totalAskVol' + str(i-1)] + df['askSize' + str(i)]
    for i in range(4,6):
        df['bidAskRatio' + str(i)] = df['totalBidVol' + str(i)] / df['totalAskVol' + str(i)]
    df['OIR'] = (df.bidSize0 - df.askSize0)/(df.bidSize0 + df.askSize0)
    lags = [*np.arange(6,10), 10]
    def addTimeFeatures(i):
        df['daskRate' + str(i)] = df.askRate0.diff(i)
        df['dbidRate' + str(i)] = df.bidRate0.diff(i)
    for i in lags:
        addTimeFeatures(i)
    df['daskRate20'] = df.askRate0.diff(20)
    df.fillna(0, inplace=True)

    df['time'] = pd.date_range(start='1/1/1970', periods=len(df), freq='T')
    df.set_index('time', inplace=True)
    df_mid = df.midRate.resample('15Min').ohlc()
    df_mid['others_dlr'] = ta.others.daily_log_return(df_mid.close, fillna=True)
    df = df.join(df_mid['others_dlr']).ffill().astype('float32')
    df.fillna(0, inplace=True)
    return df, df_mid

In [ ]:
rlcv = RollingWindowSplit(n_splits=5, compatible=True)

In [ ]:
limited_train, df_mid_train = create_limited_features_orig(train_df)

In [ ]:
# undropped
x_train = limited_train.drop(limited_train.columns[:61], axis=1).values
y_train = limited_train.y.values

# standardise
scaler = StandardScaler()
scaler.fit(x_train)
x_scaled_train = scaler.transform(x_train)

In [ ]:
# takes 11s on limited variables, 1 min on pca variables, 16m21s on 232 non-pca variables
lasso = LassoLarsCV(cv=rlcv, n_jobs=-1).fit(x_scaled_train, y_train)

In [14]:
# dump(scaler, 'scaler_limited.joblib')
# dump(lasso, 'lasso_limited.joblib')
scaler = load('scaler_limited.joblib')
lasso = load('lasso_limited.joblib')

In [ ]:
def score(model):
    limited_test, df_mid_test = create_limited_features_orig(test_df)
    x_test = limited_test.drop(limited_test.columns[:61], axis=1).values
    y_test = limited_test.y.values
    x_scaled_test = scaler.transform(x_test)
    predictions = lasso.predict(x_scaled_test)
    test_score = r2d2(y_test, predictions)
    train_score = lasso.score(x_scaled_train, y_train)
    print(f'{train_score:.4f}, {test_score:.4f}')
    return predictions, limited_test, df_mid_test

In [ ]:
predictions, limited_test, df_mid_test = score(lasso)

In [ ]:
def manual_score(model):
    x_test = test_df[test_df.columns[61:]].values
    y_test = test_df.y.values
    x_scaled_test = scaler.transform(x_test)
    test_preds = x_scaled_test @ lasso.coef_
    print(f'{r2d2(y_test, test_preds):.4f}')

In [ ]:
manual_score(lasso) # this is the score if i create the datasets first

In [ ]:
# dump(lasso, 'lasso_limited.joblib')
scaler = load('scaler_full.joblib')
lasso = load('lasso_full.joblib')

In [ ]:
lassocoef = lasso.coef_[np.where(lasso.coef_)]; lassocoef

In [ ]:
coefs = np.insert(np.insert(lassocoef,-2,np.zeros(2)),0,np.zeros(10))

In [ ]:
new_df.columns[61:][np.where(lasso.coef_)]

# Fast.ai

In [ ]:
dep_var = 'y'
procs = [FillMissing, Normalize]

In [ ]:
path = f'D:\Coding\XTX Forecasting Challenge'
data = TabularDataBunch.from_df(path = path, df = df[:int(5e5)], dep_var = 'y', procs=procs,
                                 valid_idx = list(range(int(4e5),int(5e5))))

In [ ]:
data.show_batch(rows=10)

In [ ]:
# data = (TabularList.from_df(df[:int(5e5)], cont_names=df.columns, procs=procs)
#                            .split_by_idx(list(range(int(0.8*5e5),int(5e5))))
#                            .label_from_df(cols=dep_var, label_cls=FloatList)
#                            .databunch())

In [ ]:
learn = tabular_learner(data, layers=[500,200], metrics=r2_score, ps=[0.001,0.01], emb_drop=0.04)

In [ ]:
learn.model

In [ ]:
learn.lr_find(end_lr=1e1)

In [ ]:
learn.recorder.plot()

In [ ]:
# model above has already diverged, we will restart.

In [ ]:
learn.fit_one_cycle(3, 1e-4, wd=0.1)

In [ ]:
learn.recorder.plot_lr(show_moms=True)

In [ ]:
learn.save('new_fastai')

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.predict(df.iloc[int(8.1e5)])

In [ ]:
df.y.iloc[int(8.1e5)]

In [ ]:
preds = learn.get_preds()

# Submission testing

In [5]:
def get_next_data_as_df(iteration):
    return pd.DataFrame([df.tail(100).iloc[iteration][:60].values])
#     return pd.DataFrame([test_df.iloc[iteration][:60].values])

In [6]:
def create_limited_features(df):
    df.columns = [*askRateList, *askSizeList, *bidRateList, *bidSizeList]
    df['midRate'] = (df.bidRate0 + df.askRate0) / 2 # necessary for ohlc
    df['bidAskVol'] = df.bidSize0 + df.askSize0 # necessary only for volume_adi
    df['totalBidVol1'] = df.bidSize0 + df.bidSize1
    df['totalAskVol1'] = df.askSize0 + df.askSize1
    for i in range(2,6):
        df['totalBidVol' + str(i)] = df['totalBidVol' + str(i-1)] + df['bidSize' + str(i)]
        df['totalAskVol' + str(i)] = df['totalAskVol' + str(i-1)] + df['askSize' + str(i)]
    for i in range(4,6):
        df['bidAskRatio' + str(i)] = df['totalBidVol' + str(i)] / df['totalAskVol' + str(i)]
    return df

In [7]:
def append_to_df(massive_df, row):
    try: row.index = [massive_df.index[-1] + timedelta(minutes=1)]
    except IndexError: row.index = [datetime(1970,1,1)]
    return massive_df.append(row, sort=False)

In [8]:
def add_time_features(df):
    df['OIR'] = (df.bidSize0 - df.askSize0)/(df.bidSize0 + df.askSize0)
    lags = [*np.arange(6,10), 10]
    def addManualTimeFeatures(i):
        df['daskRate' + str(i)] = df.askRate0.diff(i)
        df['dbidRate' + str(i)] = df.bidRate0.diff(i)
    for i in lags:
        addManualTimeFeatures(i)
    df['daskRate20'] = df.askRate0.diff(20)
    df.fillna(0, inplace=True)
    return df[-25:]

In [9]:
def add_resample_features(massive_df, resampled_df):
    leftovers = (massive_df.index[-1].to_pydatetime().minute+1) % 15
    def pad_history():
        full_resampled = resampled_df.append(row_ohlcv, sort=False)
        a = pd.DataFrame([full_resampled.iloc[0] for j in range(1+1-len(full_resampled))])
        a = a.append(full_resampled, sort=False)
        a.index = pd.date_range(start=row_ohlcv.index[-1], periods=len(a), freq='-15Min').sort_values()
        full_resampled['others_dlr'] = ta.others.daily_log_return(a.close, fillna=True)
        return full_resampled
    if leftovers == 0:
        row_ohlcv = massive_df.tail(15).midRate.resample('15Min').ohlc().tail(1)
        full_resampled = pad_history()
        resampled_df = resampled_df.append(full_resampled, sort=False).tail(2) # take last 2 only
    else:
        row_ohlcv = massive_df.tail(leftovers).midRate.resample('15Min').ohlc().tail(1)
        full_resampled = pad_history()
    try: massive_df.drop('others_dlr', axis=1, inplace=True)
    except KeyError: pass
    massive_df = massive_df.join(full_resampled['others_dlr'])
    massive_df = massive_df.ffill().astype('float32')
    return massive_df, resampled_df

In [10]:
def get_prediction(data):
    X = data.values
    X_scaled = scaler.transform(X)
    return np.clip(lasso.predict(np.atleast_2d(X_scaled)), -5, 5)[0]

In [ ]:
massive_df, resampled_df, log_data = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
predictions = []

In [ ]:
for iteration in tqdm_notebook(range(30)):    
    base_row = get_next_data_as_df(iteration)
    row = create_limited_features(base_row)
    massive_df = append_to_df(massive_df, row)
    massive_df = add_time_features(massive_df)
    massive_df, resampled_df = add_resample_features(massive_df, resampled_df)
    log_data_row = pd.DataFrame([massive_df.iloc[-1]]) # for debug
    data = pd.DataFrame([massive_df.iloc[-1][relevant]])
    prediction = get_prediction(data)
    predictions.append(prediction)
    log_data = log_data.append(log_data_row, sort=False)  # for debug

In [ ]:
log_data

In [ ]:
massive_df

In [11]:
def true_rlcvscore(test_df, length):
    massive_df, resampled_df = pd.DataFrame(), pd.DataFrame()
    predictions = []
    log_data = pd.DataFrame()  # for debug
    for iteration in tqdm_notebook(range(length)):    
#     for iteration in tqdm_notebook(range(len(test_df))):
        base_row = get_next_data_as_df(iteration)
        row = create_limited_features(base_row)
        massive_df = append_to_df(massive_df, row)
        massive_df = add_time_features(massive_df)
        massive_df, resampled_df = add_resample_features(massive_df, resampled_df)
        log_data_row = pd.DataFrame([massive_df.iloc[-1]]) # for debug
        data = pd.DataFrame([massive_df.iloc[-1][relevant]])
        prediction = get_prediction(data)
        predictions.append(prediction)
        log_data = log_data.append(log_data_row, sort=False)  # for debug
    score = r2d2(test_df.y.head(length), predictions)
    print(score)
    return predictions, score, log_data

In [15]:
test_predictions, test_score, log_data = true_rlcvscore(test_df, 100)


-0.08460555600257247


In [ ]:
log_data[['midRate', 'others_dlr']]

In [ ]:
limited_test.drop('y', axis=1)[['midRate', 'others_dlr']]

In [ ]:
log_data == limited_test.drop('y', axis=1)

In [ ]:
# why is logdata.others_dlr only change at row 22, but limited_test changes only at row 18?
# most probably because others_dlr uses closing price, and closing price at 22 drops to 1583.75 probably

In [ ]:
# consider changing the ohlc window length, increasing it will reduce the number of mistakes
# but increasing it also reduces the speed of reaction
# try with different values

In [ ]:
df_mid_test

In [ ]:
# %lprun -f true_rlcvscore test_predictions, test_score = true_rlcvscore(test_df, 100)